In [ ]:
import pandas as pd
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
import time

# Dataset yükləndi
df = pd.read_csv('/content/raw_data.csv')

# Unikal ölkələri tap
countries = df['Area'].unique()

# Nominatim obyektini yarat
geolocator = Nominatim(user_agent="crop_prediction_app_v2", timeout=10)

# Saniyədə maksimum 1 sorğu göndərmək üçün RateLimiter
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1.5)

country_coords = {}

print(f"{len(countries)} ölkə üçün koordinatlar tapılır...")

for country in countries:
    try:
        # Sorğu göndər
        location = geocode(country)
        if location:
            country_coords[country] = (location.latitude, location.longitude)
            print(f"Tapıldı: {country} -> {location.latitude}, {location.longitude}")
        else:
            country_coords[country] = (None, None)
            print(f"Tapılmadı: {country}")
    except Exception as e:
        print(f"Xəta baş verdi ({country}): {e}")
        country_coords[country] = (None, None)
        time.sleep(2) # Xəta olduqda fasile

# Koordinatlar datasetə əlavə olunur
df['lat'] = df['Area'].map(lambda x: country_coords.get(x, (None, None))[0])
df['lon'] = df['Area'].map(lambda x: country_coords.get(x, (None, None))[1])

print("Əməliyyat tamamlandı!")

101 ölkə üçün koordinatlar tapılır...
Tapıldı: Albania -> 41.000028, 19.9999619
Tapıldı: Algeria -> 28.0000272, 2.9999825
Tapıldı: Angola -> -11.8775768, 17.5691241
Tapıldı: Argentina -> -34.9964963, -64.9672817
Tapıldı: Armenia -> 40.7696272, 44.6736646
Tapıldı: Australia -> -24.7761086, 134.755
Tapıldı: Austria -> 47.59397, 14.12456
Tapıldı: Azerbaijan -> 40.3936294, 47.7872508
Tapıldı: Bahamas -> 24.7736546, -78.0000547
Tapıldı: Bahrain -> 26.1551249, 50.5344606
Tapıldı: Bangladesh -> 24.4769288, 90.2934413
Tapıldı: Belarus -> 53.4250605, 27.6971358
Tapıldı: Belgium -> 50.6402809, 4.6667145
Tapıldı: Botswana -> -23.1681782, 24.5928742
Tapıldı: Brazil -> -10.3333333, -53.2
Tapıldı: Bulgaria -> 42.6073975, 25.4856617
Tapıldı: Burkina Faso -> 12.0753083, -1.6880314
Tapıldı: Burundi -> -3.426449, 29.9324519
Tapıldı: Cameroon -> 4.6125522, 13.1535811
Tapıldı: Canada -> 61.0666922, -107.991707
Tapıldı: Central African Republic -> 7.0323598, 19.9981227
Tapıldı: Chile -> -31.7613365, -71.31

In [ ]:
# Unikal Ölkə və İl cütlərini (hədəfləri) yaradırıq
weather_targets = df[['Area', 'Year', 'lat', 'lon']].drop_duplicates().reset_index(drop=True)

print(f"Hazırdır! {len(weather_targets)} fərqli nöqtə üçün hava məlumatı çəkiləcək.")

Hazırdır! 2250 fərqli nöqtə üçün hava məlumatı çəkiləcək.


In [ ]:
import requests
def get_weather_data(lat, lon, year):
    if pd.isna(lat) or pd.isna(lon):
        return None, None, None

    # URL-i yoxla
    url = (f"https://archive-api.open-meteo.com/v1/archive?"
           f"latitude={float(lat)}&longitude={float(lon)}&"
           f"start_date={year}-01-01&end_date={year}-12-31&"
           f"daily=temperature_2m_mean,rain_sum,snowfall_sum&timezone=auto")

    try:
        response = requests.get(url)

        # Əgər 400 və ya 429 xətası alırıqsa, səbəbi çap edək
        if response.status_code != 200:
            print(f"Xəta: {row['Area']} ({year}) üçün Status Code: {response.status_code}")
            print(response.text) # API-dan gələn xəta mesajı
            return None, None, None

        data = response.json()

        if 'daily' not in data:
            print(f"Məlumat tapılmadı: {row['Area']} ({year})")
            return None, None, None

        daily = data['daily']

        # Boş siyahıları yoxlayaq
        avg_temp = round(sum(daily['temperature_2m_mean']) / len(daily['temperature_2m_mean']), 2) if daily.get('temperature_2m_mean') else None
        total_rain = round(sum(daily['rain_sum']), 2) if daily.get('rain_sum') else None
        total_snow = round(sum(daily['snowfall_sum']), 2) if daily.get('snowfall_sum') else None

        return avg_temp, total_rain, total_snow

    except Exception as e:
        print(f"Sistem xətası: {e}")
        return None, None, None


In [ ]:
from google.colab import files # Kompyuterə yükləmək üçün

def get_weather_data(lat, lon, year):
    if pd.isna(lat) or pd.isna(lon):
        return None, None, None
    url = (f"https://archive-api.open-meteo.com/v1/archive?"
           f"latitude={lat}&longitude={lon}&"
           f"start_date={year}-01-01&end_date={year}-12-31&"
           f"daily=temperature_2m_mean,precipitation_sum,snowfall_sum&"
           f"timezone=auto")
    try:
        response = requests.get(url, timeout=10)
        if response.status_code == 200:
            data = response.json()
            daily = data.get('daily', {})
            # Ortalama Temperatur
            avg_t = round(sum(daily.get('temperature_2m_mean', [])) / len(daily.get('temperature_2m_mean', [])), 2) if daily.get('temperature_2m_mean') else None
            # Cəmi Yağıntı
            total_p = round(sum(daily.get('precipitation_sum', [])), 2) if daily.get('precipitation_sum') else None
            # Cəmi Qar
            total_s = round(sum(daily.get('snowfall_sum', [])), 2) if daily.get('snowfall_sum') else None
            return avg_t, total_p, total_s
        return None, None, None
    except:
        return None, None, None

start = 0
end = 500
current_batch = weather_targets.iloc[start:end]
batch_results = []

print(f"{start} ilə {end} arası məlumatlar çəkilir...")

for index, row in current_batch.iterrows():
    t, p, s = get_weather_data(row['lat'], row['lon'], row['Year'])
    batch_results.append({
        'Area': row['Area'], 'Year': row['Year'],
        'avg_temp': t, 'total_precip': p, 'total_snow': s
    })

    if index % 10 == 0:
        print(f"Sətir {index} tamamlandı: {row['Area']} ({row['Year']})")

    # 2.5 saniyə fasilə
    time.sleep(2.5)

# yadda saxla və yüklə
filename = f"weather_data_{start}_{end}.csv"
batch_df = pd.DataFrame(batch_results)
batch_df.to_csv(filename, index=False)
files.download(filename) # Faylı avtomatik kompyuterinə yükləyir

print(f"Tamamlandı! {filename} faylı kompyuterinə yükləndi.")

0 ilə 500 arası məlumatlar çəkilir...
Sətir 0 tamamlandı: Albania (1990)
Sətir 10 tamamlandı: Albania (2000)
Sətir 20 tamamlandı: Albania (2011)
Sətir 30 tamamlandı: Algeria (1997)
Sətir 40 tamamlandı: Algeria (2008)
Sətir 50 tamamlandı: Angola (1994)
Sətir 60 tamamlandı: Angola (2005)
Sətir 70 tamamlandı: Argentina (1991)
Sətir 80 tamamlandı: Argentina (2001)
Sətir 90 tamamlandı: Argentina (2012)
Sətir 100 tamamlandı: Armenia (2000)
Sətir 110 tamamlandı: Armenia (2011)
Sətir 120 tamamlandı: Australia (1997)
Sətir 130 tamamlandı: Australia (2008)
Sətir 140 tamamlandı: Austria (1994)
Sətir 150 tamamlandı: Austria (2005)
Sətir 160 tamamlandı: Azerbaijan (1993)
Sətir 170 tamamlandı: Azerbaijan (2004)
Sətir 180 tamamlandı: Bahamas (1992)
Sətir 190 tamamlandı: Bahamas (2002)
Sətir 200 tamamlandı: Bahamas (2013)
Sətir 210 tamamlandı: Bahrain (1999)
Sətir 220 tamamlandı: Bahrain (2010)
Sətir 230 tamamlandı: Bangladesh (1996)
Sətir 240 tamamlandı: Bangladesh (2007)
Sətir 250 tamamlandı: Belaru

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Tamamlandı! weather_data_0_500.csv faylı kompyuterinə yükləndi.


In [ ]:
# Köhnə siyahını boşalt ki, yeni hissə üçün yer açılsın
batch_results = []

In [ ]:
from google.colab import files # Kompyuterə yükləmək üçün

def get_weather_data(lat, lon, year):
    if pd.isna(lat) or pd.isna(lon):
        return None, None, None
    url = (f"https://archive-api.open-meteo.com/v1/archive?"
           f"latitude={lat}&longitude={lon}&"
           f"start_date={year}-01-01&end_date={year}-12-31&"
           f"daily=temperature_2m_mean,precipitation_sum,snowfall_sum&"
           f"timezone=auto")
    try:
        response = requests.get(url, timeout=10)
        if response.status_code == 200:
            data = response.json()
            daily = data.get('daily', {})
            # Ortalama Temperatur
            avg_t = round(sum(daily.get('temperature_2m_mean', [])) / len(daily.get('temperature_2m_mean', [])), 2) if daily.get('temperature_2m_mean') else None
            # Cəmi Yağıntı
            total_p = round(sum(daily.get('precipitation_sum', [])), 2) if daily.get('precipitation_sum') else None
            # Cəmi Qar
            total_s = round(sum(daily.get('snowfall_sum', [])), 2) if daily.get('snowfall_sum') else None
            return avg_t, total_p, total_s
        return None, None, None
    except:
        return None, None, None


start = 500
end = 1000
current_batch = weather_targets.iloc[start:end]
batch_results = []

print(f"{start} ilə {end} arası məlumatlar çəkilir...")

for index, row in current_batch.iterrows():
    t, p, s = get_weather_data(row['lat'], row['lon'], row['Year'])
    batch_results.append({
        'Area': row['Area'], 'Year': row['Year'],
        'avg_temp': t, 'total_precip': p, 'total_snow': s
    })

    if index % 10 == 0:
        print(f"Sətir {index} tamamlandı: {row['Area']} ({row['Year']})")

    #  2.5 saniyə fasilə
    time.sleep(2.5)

# yadda saxla və yüklə
filename = f"weather_data_{start}_{end}.csv"
batch_df = pd.DataFrame(batch_results)
batch_df.to_csv(filename, index=False)
files.download(filename) # Faylı avtomatik kompyuterinə yükləyir

print(f"Tamamlandı! {filename} faylı kompyuterinə yükləndi.")

500 ilə 1000 arası məlumatlar çəkilir...
Sətir 500 tamamlandı: Colombia (2002)
Sətir 510 tamamlandı: Colombia (2013)
Sətir 520 tamamlandı: Croatia (2001)
Sətir 530 tamamlandı: Croatia (2012)
Sətir 540 tamamlandı: Denmark (1994)
Sətir 550 tamamlandı: Denmark (2005)
Sətir 560 tamamlandı: Dominican Republic (1995)
Sətir 570 tamamlandı: Dominican Republic (2006)
Sətir 580 tamamlandı: Ecuador (1992)
Sətir 590 tamamlandı: Ecuador (2002)
Sətir 600 tamamlandı: Ecuador (2013)
Sətir 610 tamamlandı: Egypt (1999)
Sətir 620 tamamlandı: Egypt (2010)
Sətir 630 tamamlandı: El Salvador (1996)
Sətir 640 tamamlandı: El Salvador (2007)
Sətir 650 tamamlandı: Eritrea (1996)
Sətir 660 tamamlandı: Eritrea (2007)
Sətir 670 tamamlandı: Estonia (1995)
Sətir 680 tamamlandı: Estonia (2006)
Sətir 690 tamamlandı: Finland (1992)
Sətir 700 tamamlandı: Finland (2002)
Sətir 710 tamamlandı: Finland (2013)
Sətir 720 tamamlandı: France (1999)
Sətir 730 tamamlandı: France (2010)
Sətir 740 tamamlandı: Germany (1996)
Sətir 75

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Tamamlandı! weather_data_500_1000.csv faylı kompyuterinə yükləndi.


In [ ]:
batch_results = []

In [ ]:
from google.colab import files # Kompyuterə yükləmək üçün

def get_weather_data(lat, lon, year):
    if pd.isna(lat) or pd.isna(lon):
        return None, None, None
    url = (f"https://archive-api.open-meteo.com/v1/archive?"
           f"latitude={lat}&longitude={lon}&"
           f"start_date={year}-01-01&end_date={year}-12-31&"
           f"daily=temperature_2m_mean,precipitation_sum,snowfall_sum&"
           f"timezone=auto")
    try:
        response = requests.get(url, timeout=10)
        if response.status_code == 200:
            data = response.json()
            daily = data.get('daily', {})
            # Ortalama Temperatur
            avg_t = round(sum(daily.get('temperature_2m_mean', [])) / len(daily.get('temperature_2m_mean', [])), 2) if daily.get('temperature_2m_mean') else None
            # Cəmi Yağıntı
            total_p = round(sum(daily.get('precipitation_sum', [])), 2) if daily.get('precipitation_sum') else None
            # Cəmi Qar
            total_s = round(sum(daily.get('snowfall_sum', [])), 2) if daily.get('snowfall_sum') else None
            return avg_t, total_p, total_s
        return None, None, None
    except:
        return None, None, None

# --- PARAMETRLƏR ---
start = 1000
end = 1500
current_batch = weather_targets.iloc[start:end]
batch_results = []

print(f"{start} ilə {end} arası məlumatlar çəkilir...")

for index, row in current_batch.iterrows():
    t, p, s = get_weather_data(row['lat'], row['lon'], row['Year'])
    batch_results.append({
        'Area': row['Area'], 'Year': row['Year'],
        'avg_temp': t, 'total_precip': p, 'total_snow': s
    })

    if index % 10 == 0:
        print(f"Sətir {index} tamamlandı: {row['Area']} ({row['Year']})")

    # 2.5 saniyə fasilə
    time.sleep(2.5)

# yadda saxla və yüklə
filename = f"weather_data_{start}_{end}.csv"
batch_df = pd.DataFrame(batch_results)
batch_df.to_csv(filename, index=False)
files.download(filename) # Faylı avtomatik kompyuterinə yükləyir

print(f"Tamamlandı! {filename} faylı kompyuterinə yükləndi.")

1000 ilə 1500 arası məlumatlar çəkilir...
Sətir 1000 tamamlandı: Iraq (2004)
Sətir 1010 tamamlandı: Ireland (1990)
Sətir 1020 tamamlandı: Ireland (2000)
Sətir 1030 tamamlandı: Ireland (2011)
Sətir 1040 tamamlandı: Italy (1997)
Sətir 1050 tamamlandı: Italy (2008)
Sətir 1060 tamamlandı: Jamaica (1994)
Sətir 1070 tamamlandı: Jamaica (2005)
Sətir 1080 tamamlandı: Japan (1991)
Sətir 1090 tamamlandı: Japan (2001)
Sətir 1100 tamamlandı: Japan (2012)
Sətir 1110 tamamlandı: Kazakhstan (2000)
Sətir 1120 tamamlandı: Kazakhstan (2011)
Sətir 1130 tamamlandı: Kenya (1997)
Sətir 1140 tamamlandı: Kenya (2008)
Sətir 1150 tamamlandı: Latvia (1996)
Sətir 1160 tamamlandı: Latvia (2007)
Sətir 1170 tamamlandı: Lebanon (1993)
Sətir 1180 tamamlandı: Lebanon (2004)
Sətir 1190 tamamlandı: Lesotho (1990)
Sətir 1200 tamamlandı: Lesotho (2000)
Sətir 1210 tamamlandı: Lesotho (2011)
Sətir 1220 tamamlandı: Libya (1997)
Sətir 1230 tamamlandı: Libya (2008)
Sətir 1240 tamamlandı: Lithuania (1996)
Sətir 1250 tamamlandı: 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Tamamlandı! weather_data_1000_1500.csv faylı kompyuterinə yükləndi.


In [ ]:
batch_results = []

In [ ]:
from google.colab import files # Kompyuterə yükləmək üçün

def get_weather_data(lat, lon, year):
    if pd.isna(lat) or pd.isna(lon):
        return None, None, None
    url = (f"https://archive-api.open-meteo.com/v1/archive?"
           f"latitude={lat}&longitude={lon}&"
           f"start_date={year}-01-01&end_date={year}-12-31&"
           f"daily=temperature_2m_mean,precipitation_sum,snowfall_sum&"
           f"timezone=auto")
    try:
        response = requests.get(url, timeout=10)
        if response.status_code == 200:
            data = response.json()
            daily = data.get('daily', {})
            # Ortalama Temperatur
            avg_t = round(sum(daily.get('temperature_2m_mean', [])) / len(daily.get('temperature_2m_mean', [])), 2) if daily.get('temperature_2m_mean') else None
            # Cəmi Yağıntı
            total_p = round(sum(daily.get('precipitation_sum', [])), 2) if daily.get('precipitation_sum') else None
            # Cəmi Qar
            total_s = round(sum(daily.get('snowfall_sum', [])), 2) if daily.get('snowfall_sum') else None
            return avg_t, total_p, total_s
        return None, None, None
    except:
        return None, None, None

start = 1500
end = 2000
current_batch = weather_targets.iloc[start:end]
batch_results = []

print(f"{start} ilə {end} arası məlumatlar çəkilir...")

for index, row in current_batch.iterrows():
    t, p, s = get_weather_data(row['lat'], row['lon'], row['Year'])
    batch_results.append({
        'Area': row['Area'], 'Year': row['Year'],
        'avg_temp': t, 'total_precip': p, 'total_snow': s
    })

    if index % 10 == 0:
        print(f"Sətir {index} tamamlandı: {row['Area']} ({row['Year']})")

    # 2.5 saniyə fasilə
    time.sleep(2.5)

# yadda saxla və yüklə
filename = f"weather_data_{start}_{end}.csv"
batch_df = pd.DataFrame(batch_results)
batch_df.to_csv(filename, index=False)
files.download(filename) # Faylı avtomatik kompyuterinə yükləyir

print(f"Tamamlandı! {filename} faylı kompyuterinə yükləndi.")

1500 ilə 2000 arası məlumatlar çəkilir...
Sətir 1500 tamamlandı: Nepal (1995)
Sətir 1510 tamamlandı: Nepal (2006)
Sətir 1520 tamamlandı: Netherlands (1992)
Sətir 1530 tamamlandı: Netherlands (2002)
Sətir 1540 tamamlandı: Netherlands (2013)
Sətir 1550 tamamlandı: New Zealand (1999)
Sətir 1560 tamamlandı: New Zealand (2010)
Sətir 1570 tamamlandı: Nicaragua (1996)
Sətir 1580 tamamlandı: Nicaragua (2007)
Sətir 1590 tamamlandı: Niger (1993)
Sətir 1600 tamamlandı: Niger (2004)
Sətir 1610 tamamlandı: Norway (1990)
Sətir 1620 tamamlandı: Norway (2000)
Sətir 1630 tamamlandı: Norway (2011)
Sətir 1640 tamamlandı: Pakistan (1997)
Sətir 1650 tamamlandı: Pakistan (2008)
Sətir 1660 tamamlandı: Papua New Guinea (1994)
Sətir 1670 tamamlandı: Papua New Guinea (2005)
Sətir 1680 tamamlandı: Peru (1991)
Sətir 1690 tamamlandı: Peru (2001)
Sətir 1700 tamamlandı: Peru (2012)
Sətir 1710 tamamlandı: Poland (1998)
Sətir 1720 tamamlandı: Poland (2009)
Sətir 1730 tamamlandı: Portugal (1995)
Sətir 1740 tamamlandı: 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Tamamlandı! weather_data_1500_2000.csv faylı kompyuterinə yükləndi.


In [ ]:
batch_results = []

In [ ]:
from google.colab import files # Kompyuterə yükləmək üçün

def get_weather_data(lat, lon, year):
    if pd.isna(lat) or pd.isna(lon):
        return None, None, None
    url = (f"https://archive-api.open-meteo.com/v1/archive?"
           f"latitude={lat}&longitude={lon}&"
           f"start_date={year}-01-01&end_date={year}-12-31&"
           f"daily=temperature_2m_mean,precipitation_sum,snowfall_sum&"
           f"timezone=auto")
    try:
        response = requests.get(url, timeout=10)
        if response.status_code == 200:
            data = response.json()
            daily = data.get('daily', {})
            # Ortalama Temperatur
            avg_t = round(sum(daily.get('temperature_2m_mean', [])) / len(daily.get('temperature_2m_mean', [])), 2) if daily.get('temperature_2m_mean') else None
            # Cəmi Yağıntı
            total_p = round(sum(daily.get('precipitation_sum', [])), 2) if daily.get('precipitation_sum') else None
            # Cəmi Qar
            total_s = round(sum(daily.get('snowfall_sum', [])), 2) if daily.get('snowfall_sum') else None
            return avg_t, total_p, total_s
        return None, None, None
    except:
        return None, None, None


start = 2000
end = 2250
current_batch = weather_targets.iloc[start:end]
batch_results = []

print(f"{start} ilə {end} arası məlumatlar çəkilir...")

for index, row in current_batch.iterrows():
    t, p, s = get_weather_data(row['lat'], row['lon'], row['Year'])
    batch_results.append({
        'Area': row['Area'], 'Year': row['Year'],
        'avg_temp': t, 'total_precip': p, 'total_snow': s
    })

    if index % 10 == 0:
        print(f"Sətir {index} tamamlandı: {row['Area']} ({row['Year']})")

    # 2.5 saniyə fasilə
    time.sleep(2.5)

# yadda saxla və yüklə
filename = f"weather_data_{start}_{end}.csv"
batch_df = pd.DataFrame(batch_results)
batch_df.to_csv(filename, index=False)
files.download(filename) # Faylı avtomatik kompyuterinə yükləyir

print(f"Tamamlandı! {filename} faylı kompyuterinə yükləndi.")

2000 ilə 2250 arası məlumatlar çəkilir...
Sətir 2000 tamamlandı: Sweden (2013)
Sətir 2010 tamamlandı: Switzerland (1999)
Sətir 2020 tamamlandı: Switzerland (2010)
Sətir 2030 tamamlandı: Tajikistan (1998)
Sətir 2040 tamamlandı: Tajikistan (2009)
Sətir 2050 tamamlandı: Thailand (1995)
Sətir 2060 tamamlandı: Thailand (2006)
Sətir 2070 tamamlandı: Tunisia (1992)
Sətir 2080 tamamlandı: Tunisia (2002)
Sətir 2090 tamamlandı: Tunisia (2013)
Sətir 2100 tamamlandı: Turkey (1999)
Sətir 2110 tamamlandı: Turkey (2010)
Sətir 2120 tamamlandı: Uganda (1996)
Sətir 2130 tamamlandı: Uganda (2007)
Sətir 2140 tamamlandı: Ukraine (1995)
Sətir 2150 tamamlandı: Ukraine (2006)
Sətir 2160 tamamlandı: United Kingdom (1992)
Sətir 2170 tamamlandı: United Kingdom (2002)
Sətir 2180 tamamlandı: United Kingdom (2013)
Sətir 2190 tamamlandı: Uruguay (1999)
Sətir 2200 tamamlandı: Uruguay (2010)
Sətir 2210 tamamlandı: Zambia (1996)
Sətir 2220 tamamlandı: Zambia (2007)
Sətir 2230 tamamlandı: Zimbabwe (1993)
Sətir 2240 tama

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Tamamlandı! weather_data_2000_2250.csv faylı kompyuterinə yükləndi.


In [ ]:
import glob

# 1. Bütün hissə-hissə yüklənən faylları bir siyahıya yığırıq
all_files = glob.glob("weather_data_*.csv")
print(f"Tapılan fayl sayı: {len(all_files)}")

# 2. Onları bir cədvəldə birləşdiririk
combined_weather = pd.concat([pd.read_csv(f) for f in all_files]).drop_duplicates().reset_index(drop=True)

# 3. Hava məlumatlarını ana datasetin (df) üzərinə 'Area' və 'Year' sütunlarına görə join edirik
final_df = pd.merge(df, combined_weather, on=['Area', 'Year'], how='left')

# 4. Yoxlayaq: Boş qalan (NaN) hava məlumatı varmı?
missing_count = final_df['avg_temp'].isnull().sum()
print(f"Hava məlumatı tapılmayan sətir sayı: {missing_count}")

# 5. YEKUN faylı kompyuterinə yüklə
final_df.to_csv("Crop_yield_with_weather.csv", index=False)
from google.colab import files
files.download("Crop_yield_with_weather.csv")

print("Hər şey hazır! Artıq bütün dünya üzrə hava və məhsuldarlıq məlumatları bir fayldadır.")

Tapılan fayl sayı: 5
Hava məlumatı tapılmayan sətir sayı: 13191


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Hər şey hazır! Artıq bütün dünya üzrə hava və məhsuldarlıq məlumatları bir fayldadır.


In [ ]:
final_df.head()

,Unnamed: 0,Area,Item,Year,hg/ha_yield,pesticides_tonnes,lat,lon,avg_temp,total_precip,total_snow
0,0,Albania,Maize,1990,36613,121.0,41.000028,19.999962,14.74,658.7,21.98
1,1,Albania,Potatoes,1990,66667,121.0,41.000028,19.999962,14.74,658.7,21.98
2,2,Albania,"Rice, paddy",1990,23333,121.0,41.000028,19.999962,14.74,658.7,21.98
3,3,Albania,Sorghum,1990,12500,121.0,41.000028,19.999962,14.74,658.7,21.98
4,4,Albania,Soybeans,1990,7000,121.0,41.000028,19.999962,14.74,658.7,21.98
